### **Case Study: Multi-Objective Optimization of a Heat Sink**

#### **Background**

You are a thermal engineer tasked with designing a passive heat sink for a new high-performance microprocessor. The heat sink's purpose is to dissipate heat generated by the chip into the surrounding air, preventing it from overheating. Your design must be both effective at cooling and economical in terms of material usage. This creates a classic engineering trade-off: a larger, heavier heat sink cools better but costs more and takes up more space.

You will use the **MOEA/D algorithm** to find the optimal geometric parameters for the heat sink that represent the best possible trade-offs between thermal performance and material cost (represented by mass).

### **Problem Description**

The heat sink is a rectangular base of fixed width $W$ and length $L$, with $N$ rectangular fins. You need to optimize three continuous design variables:

* $x_1$: Fin Height ($H$) in mm
* $x_2$: Fin Thickness ($t$) in mm
* $x_3$: Number of Fins ($N$) - For simplicity in this continuous optimization problem, treat $N$ as a float and round it to the nearest integer for calculations.

**Objectives (to be minimized simultaneously):**

1.  **Thermal Resistance ($f_1$)**: Represents how effectively the heat sink dissipates heat. A lower value is better.
2.  **Mass ($f_2$)**: Represents the material cost and weight of the heat sink. A lower value is better.

**Constraints:** The design is subject to physical constraints related to fin width, aspect ratio, and the gap between fins.

**Variable Ranges:**

* $x_1$ (Fin Height, H): $[10, 40]$ mm
* $x_2$ (Fin Thickness, t): $[0.5, 3]$ mm
* $x_3$ (Number of Fins, N): $[5, 25]$

#### **Programming Requirements**

1.  **Review the Provided Code**: The functions for the objective and constraint calculations, as well as the main MOEA/D algorithm structure, are provided for you.

2.  **Configure the MOEA/D Algorithm**: Your task is to set the main parameters for the algorithm inside the `run_moead_heatsink` function. You need to provide appropriate integer or float values for:
    * `n_subproblems` (the population size)
    * `n_neighbors` (the neighborhood size)
    * `max_gen` (the number of generations)
    * `mutation_rate`

    Set these to reasonable values (e.g., `n_subproblems = 200`, `n_neighbors = 40`, `max_gen = 400`, `mutation_rate = 0.2`).

3.  **Run the Notebook**: Execute all cells to run the optimization. The autograding system will evaluate the final `Ideal point` printed in the output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Problem Definition: Heat Sink (Provided) ---

# Constants
W = 50.0
L = 50.0
T_BASE = 2.0
H_COEFF = 0.005
DENSITY = 2.7e-3
K_PENALTY = 1e7

def calculate_penalized_objectives(population):
    pop = population.copy()
    H, t, N = pop[:, 0], pop[:, 1], np.round(pop[:, 2])
    area_base_exposed = W * L - N * t * L
    area_fins = N * (2 * H * L + 2 * H * t)
    total_area = area_base_exposed + area_fins
    f1 = 1 / (H_COEFF * total_area)
    volume_base = W * L * T_BASE
    volume_fins = N * H * t * L
    f2 = DENSITY * (volume_base + volume_fins)
    g1 = N * t - W
    g2 = H / t - 15
    gap = np.full_like(N, 2.0)
    valid_gap_mask = N >= 2
    gap[valid_gap_mask] = (W - N[valid_gap_mask] * t[valid_gap_mask]) / (N[valid_gap_mask] - 1)
    g3 = 2.0 - gap
    g1[g1 < 0] = 0
    g2[g2 < 0] = 0
    g3[g3 < 0] = 0
    total_penalty = K_PENALTY * (g1 + g2 + g3)
    h1 = f1 + total_penalty
    h2 = f2 + total_penalty
    return np.column_stack((h1, h2))

def get_unpenalized_objectives(population):
    pop = population.copy()
    H, t, N = pop[:, 0], pop[:, 1], np.round(pop[:, 2])
    area_base_exposed = W * L - N * t * L
    area_fins = N * (2 * H * L + 2 * H * t)
    total_area = area_base_exposed + area_fins
    f1 = 1 / (H_COEFF * total_area)
    volume_base = W * L * T_BASE
    volume_fins = N * H * t * L
    f2 = DENSITY * (volume_base + volume_fins)
    return np.column_stack((f1, f2))

In [ ]:
# --- 2. MOEA/D Framework Implementation (Provided) ---

def Tchebycheff(fitness_values, weights, ideal_point):
    return np.max(np.abs(fitness_values - ideal_point) * weights, axis=1)

def run_moead_heatsink():
    # --- YOUR TASK: Configure the Algorithm Parameters ---
    n_subproblems = None  # YOUR CODE HERE: e.g., 200
    n_neighbors = None    # YOUR CODE HERE: e.g., 40
    max_gen = None        # YOUR CODE HERE: e.g., 400
    mutation_rate = None  # YOUR CODE HERE: e.g., 0.2
    # --- End of Your Task ---

    # Check if parameters are set
    if any(p is None for p in [n_subproblems, n_neighbors, max_gen, mutation_rate]):
        print("Algorithm parameters not set. Please complete the configuration.")
        return None

    dim = 3
    x_min = np.array([10.0, 0.5, 5.0])
    x_max = np.array([40.0, 3.0, 25.0])

    # Initialization
    lambdas = np.zeros((n_subproblems, 2))
    lambdas[:, 0] = np.linspace(0, 1, n_subproblems)
    lambdas[:, 1] = 1 - lambdas[:, 0]
    dist_matrix = np.sum((lambdas[:, np.newaxis, :] - lambdas[np.newaxis, :, :])**2, axis=2)
    neighbourhood = np.argsort(dist_matrix, axis=1)[:, :n_neighbors]
    population = x_min + np.random.rand(n_subproblems, dim) * (x_max - x_min)
    fitness = calculate_penalized_objectives(population)
    z = np.min(fitness, axis=0)

    # Evolutionary Loop
    for gen in range(max_gen):
        if gen % 40 == 0:
            print(f"Generation {gen}... Ideal point: z = [{z[0]:.3f}, {z[1]:.2f}]")
        for i in range(n_subproblems):
            parent_indices = np.random.choice(neighbourhood[i], 2, replace=False)
            p1, p2 = population[parent_indices]
            offspring = p1.copy()
            crossover_mask = np.random.rand(dim) > 0.5
            offspring[crossover_mask] = p2[crossover_mask]
            mutation_mask = np.random.rand(dim) < mutation_rate
            offspring[mutation_mask] = x_min[mutation_mask] + np.random.rand(np.sum(mutation_mask)) * (x_max[mutation_mask] - x_min[mutation_mask])
            offspring = np.clip(offspring, x_min, x_max)
            offspring_fitness = calculate_penalized_objectives(offspring.reshape(1, -1))
            z = np.minimum(z, offspring_fitness[0])
            for j in neighbourhood[i]:
                current_tcheby = Tchebycheff(fitness[j].reshape(1,-1), lambdas[j].reshape(1,-1), z)
                offspring_tcheby = Tchebycheff(offspring_fitness, lambdas[j].reshape(1,-1), z)
                if offspring_tcheby < current_tcheby:
                    population[j] = offspring
                    fitness[j] = offspring_fitness[0]
    print("Optimization finished.")
    return population

In [ ]:
# --- 3. Run and Visualize (Provided) ---
final_population = run_moead_heatsink()

if final_population is not None:
    # Calculate final unpenalized objectives for plotting
    final_objectives = get_unpenalized_objectives(final_population)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(final_objectives[:, 0], final_objectives[:, 1], c=final_objectives[:, 0], cmap='viridis_r', alpha=0.8)
    plt.colorbar(label='Thermal Resistance (°C/W)')
    plt.title('Approximated Pareto Front for Heat Sink Design (MOEA/D)', fontsize=16)
    plt.xlabel('Objective 1: Thermal Resistance ($f_1$)', fontsize=14)
    plt.ylabel('Objective 2: Mass ($f_2$)', fontsize=14)
    plt.grid(True)
    plt.show()
else:
    print("Execution halted. Please set the GA parameters to run the visualization.")